In [6]:
import random
import json
import pickle
import numpy as np
import nltk
import os
import tensorflow
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD


In [7]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('test_intents.json').read())

words = []
documents = []
classes = []
ignore_words = ['?','.','!',',','_','#']

In [8]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

[(['start'], 'start'), (['begin'], 'start'), (['hello'], 'start'), (['hi'], 'start'), (['delete', 'ticket'], 'delete_ticket'), (['remove', 'ticket'], 'delete_ticket'), (['cancel', 'ticket'], 'delete_ticket'), ([], 'delete_ticket'), (['manual', 'locate'], 'manual_locate'), (['locate'], 'manual_locate'), (['location'], 'manual_locate'), (['show', 'tickets'], 'show_tickets'), (['view', 'tickets'], 'show_tickets'), (['list', 'tickets'], 'show_tickets'), (['see', 'tickets'], 'show_tickets'), (['{', 'location_name', '}', ',', '{', 'museum_name', '}'], 'location_and_museum'), (['book', 'a', 'ticket'], 'book_ticket'), (['ticket', 'booking'], 'book_ticket'), (['I', 'want', 'to', 'book'], 'book_ticket'), (['about', 'the', 'museum'], 'about_museum'), (['museum', 'info'], 'about_museum'), (['information', 'about', 'museum'], 'about_museum'), (['description', 'of', 'the', 'museum'], 'about_museum'), (['timings'], 'museum_timings'), (['museum', 'timings'], 'museum_timings'), (['opening', 'hours'], '

In [9]:
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_words]
words=sorted(set(words))

classes=sorted(set(classes))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


In [10]:
training = []
output_empty=[0]*len(classes)

for document in documents:
    bag = []
    word_patterns=document[0]
    word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append(bag+output_row)

random.shuffle(training)
training=np.array(training)

x_train=training[:,:len(words)]
y_train=training[:,len(words):]

model=tensorflow.keras.Sequential()
model.add(tensorflow.keras.layers.Dense(128,input_shape=(len(x_train[0]),),activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.5))
model.add(tensorflow.keras.layers.Dense(64,activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.5))
model.add(tensorflow.keras.layers.Dense(len(y_train[0]),activation='softmax'))
sgd=tensorflow.keras.optimizers.SGD(learning_rate=0.01,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
hist=model.fit(x_train,y_train,epochs=200,batch_size=5,verbose=1)
model.save('chatbot_model.h5',hist)

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0482 - loss: 2.4158    
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2236 - loss: 2.2972 
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0597 - loss: 2.3663     
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1417 - loss: 2.2483     
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2289 - loss: 2.1751 
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2710 - loss: 2.2204 
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3607 - loss: 1.9942 
Epoch 8/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2466 - loss: 2.1860     
Epoch 9/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4309 - loss: 2.0142 
Epoch 10/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3795 - loss: 1.9106 
Epoch 11/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4249 - loss: 1.9922 
Epoch 12/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accurac